# **Installing and Importing Necessary Libraries**

In [1]:
!pip install -q -U transformers peft accelerate optimum

In [2]:
!pip install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch


# **Loading and Preparing a Quantized Model for Training**

In [4]:
from peft import prepare_model_for_kbit_training

model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config_loading, device_map="auto")

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [5]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.GPTQ: 'gptq'>,
 'bits': 4,
 'tokenizer': None,
 'dataset': None,
 'group_size': 128,
 'damp_percent': 0.01,
 'desc_act': False,
 'sym': True,
 'true_sequential': True,
 'use_cuda_fp16': False,
 'model_seqlen': None,
 'block_name_to_quantize': None,
 'module_name_preceding_first_block': None,
 'batch_size': 1,
 'pad_token_id': None,
 'disable_exllama': True}

In [6]:
model = prepare_model_for_kbit_training(model)

# **Configuring and Initializing LoRA Parameters for the Model**

In [7]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.10,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 270,798,848 || trainable%: 3.097726619575575


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# **Loading Dataset**

In [9]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

# **Splitting Dataset**





In [10]:
data = data["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 2006
    })
    test: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 502
    })
})

# **Defining Training Arguments and Setting Up Trainer**

In [11]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        evaluation_strategy="steps",
        eval_steps=20,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=20,
        output_dir="outputs",
        optim="adamw_hf"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20,2.255600,2.235854
40,1.652100,1.792940
60,3.399900,8.224396
80,7.929000,7.446287
100,6.389500,6.715991


TrainOutput(global_step=100, training_loss=4.325219841003418, metrics={'train_runtime': 407.0593, 'train_samples_per_second': 0.246, 'train_steps_per_second': 0.246, 'total_flos': 5477012987904.0, 'train_loss': 4.325219841003418, 'epoch': 0.05})

In [12]:


from huggingface_hub import HfApi
api = HfApi()

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
api.create_repo("DrishtiSharma/llama-2-chat-gptq-english-quotes-test-lora-alpha-16")

RepoUrl('https://huggingface.co/DrishtiSharma/llama-2-chat-gptq-english-quotes-test-lora-alpha-16', endpoint='https://huggingface.co', repo_type='model', repo_id='DrishtiSharma/llama-2-chat-gptq-english-quotes-test-lora-alpha-16')

In [18]:
api.upload_folder(
    folder_path = "/content/outputs",
    path_in_repo = ".",
    repo_id = "DrishtiSharma/llama-2-chat-gptq-english-quotes-test-lora-alpha-16",
    repo_type = "model"
                  )

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/67.3M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/627 [00:00<?, ?B/s]

Upload 12 LFS files:   0%|          | 0/12 [00:00<?, ?it/s]

...

events.out.tfevents.1694640189.b04c78a5e2b1.6813.0:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1694640653.b04c78a5e2b1.8924.0:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

events.out.tfevents.1694641228.b04c78a5e2b1.11505.0:   0%|          | 0.00/5.12k [00:00<?, ?B/s]

events.out.tfevents.1694641365.b04c78a5e2b1.12272.0:   0%|          | 0.00/7.00k [00:00<?, ?B/s]

'https://huggingface.co/DrishtiSharma/llama-2-chat-gptq-english-quotes-test-lora-alpha-16/tree/main/.'